# Airbnb Price Analyis

1. Webscrape Airbnb page to find absolute location of stay
2. Convert absolute location to address
4. Find estimated rent of the AirBNB
6. Compare Airbnb price with estimated price after utilities, analyze the Airbnb premium

### Imports

In [67]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import requests
import json
import time
from googlesearch import search

In [30]:
load_dotenv()

def get_numbers(w):
    nums = ""
    for c in w:
        if (c.isnumeric()):
            nums += c
    return (w if nums == "" else int(nums))

### Step 1: Webscraping AirBNB Rental

In [48]:
options = Options()
options.add_argument("-headless")
driver = webdriver.Firefox(options=options)

In [60]:
# Enter an Airbnb link
listing_url = "https://www.airbnb.com/rooms/22015858"
driver.get(listing_url)
time.sleep(5)
info = driver.find_element("class name","lgx66tx").text.split("·")
price = get_numbers(driver.find_elements("class name", "_tyxjp1")[1].text.strip())
map_link, scroll_down_height = None, 0
while not map_link:
    try:
        map_link = driver.find_element("xpath","/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[5]/div/div/div/div[2]/section/div[3]/div[4]/div[2]/div/div/div[16]/div/div[5]/div[2]/a").get_attribute("href")
    except:
        scroll_down_height += 20
        driver.execute_script("window.scroll(0, " + str(scroll_down_height) + ");")
info = [x.strip() for x in info]
num_guests, num_br, num_bed = (get_numbers(x) for x in info[0:3])
num_bath = int(np.ceil(float(info[3].split()[0])))
lat, lng, spec_char_count = "", "", 0
for c in map_link:
    if(c == "@" or c == ","):
        spec_char_count += 1
    elif(spec_char_count == 1):
        lat += c
    elif(spec_char_count == 2):
        lng += c
    elif(spec_char_count == 3):
        break
print("$" + str(price), "per night")
print(str(num_br) + (" Bedroom, " if num_br == 1 else (" Bedrooms, " if isinstance(num_br,int) else ", ")) + str(num_bath) + (" Bathroom" if num_bath == 1 else " Bathrooms"))
print(lat,lng,sep = ", ")

$285 per night
3 Bedrooms, 3 Bathrooms
39.50307, -79.3141


### Step 2. Getting AirBNB Address

In [61]:
loc_url = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&location=" + lng + "," + lat + "&token=" + os.getenv("API_TOKEN")
response = requests.get(loc_url)
address_json = response.json()
add_num = address_json["address"]["AddNum"]
street = (address_json["address"]["Address"] if add_num == "" else address_json["address"]["Address"][(len(add_num) + 1):])
city = address_json["address"]["City"]
state = address_json["address"]["RegionAbbr"]
zipcode = address_json["address"]["Postal"]
street_address = ((street + ", ") if street != "" else "") + city + ", " + state + " " + zipcode
address = (street_address if add_num == "" else add_num + " " + street_address)
print(address)

788 Glendale Rd, Oakland, MD 21550


### Step 3. Getting Associated Estimated Rent

In [75]:
zillow_link = [_ for _ in search(address + " zillow", num_results=1)]
driver.get(address_links[0])

print(address_links[0])

https://www.zillow.com/homedetails/788-Glendale-Rd-Oakland-MD-21550/58259958_zpid/


In [64]:

driver.get("https://app.rentcast.io/app")
elems = {"Text":[], "Element":[]}
for elem in driver.find_elements("class name", "dropdown-item"):
    elems[elem.get_attribute("textContent")] = elem
driver.find_element("class name", "form-control").send_keys(address)
driver.find_elements("class name", "dropdown-toggle")[3].click()
elems[" Apartment "].click()
driver.find_elements("class name", "dropdown-toggle")[4].click()
bedrooms = ""
elems[" " + (((str(num_br) if num_br < 6 else "6+") + (" Bed" if num_br == 1 else " Beds")) if isinstance(num_br, int) else num_br) + " "].click()
driver.find_elements("class name", "dropdown-toggle")[5].click()
elems[" " + (str(num_bath) if num_bath < 4 else "4+") + (" Bath" if num_bath == 1 else " Baths") + " "].click()
driver.find_element("class name", "btn-primary").click()
time.sleep(3)
test_addresses = []
if (add_num != ""):
    for i in range(4):
        x = int(np.ceil((i+1)/2) * pow(-1,i))
        new_add_num = int(add_num) + x
        test_addresses.append(str(new_add_num) + " " + street_address)
test_addresses.append(street_address)
test_addresses.append(city + ", " + state + " " + zipcode)
rent_est = None
for test_address in test_addresses:
    if (rent_est):
        break
    try:
        rent_est = get_numbers(driver.find_element("class name", "display-3").get_attribute("textContent"))
    except:
        driver.find_element("class name", "form-control").clear()
        driver.find_element("class name", "form-control").send_keys(test_address)
        driver.find_element("class name", "btn-primary").click()
        time.sleep(len(test_addresses) + 2)
remote_location = False
if (not rent_est):
    state_rent_data = pd.read_csv("state_rent_data.csv")
    rent_est = round(state_rent_data.loc[state_rent_data["state"] == address_json["address"]["Region"]].iloc[0,1] * (num_br + num_bath / 3))
    remote_location = True
print(rent_est)

NameError: name 'search' is not defined

### 4. Calculate Stay Premium and Host Profits


In [63]:
utl_avg = 429.33 # from https://www.forbes.com/home-improvement/living/monthly-utility-costs-by-state/
day_rent_est = round(((rent_est + utl_avg) * 12 / 365.25),2)
premium_perc = round(price/day_rent_est * 100)
host_profit = round(price - day_rent_est,2)
print("Your daily price: $" + str(price))
print("Estimated daily host costs: $" + str(day_rent_est))
print("Estimated stay premium: " + str(premium_perc) + "%")
print("Estimated daily host profit while occupied: $" + str(host_profit))
if (remote_location):
    print("Remote location detected: estimates likely to be inaccurate")

Your daily price: $285
Estimated daily host costs: $200.06
Estimated stay premium: 142%
Estimated daily host profit while occupied: $84.94
Remote location detected: estimates likely to be inaccurate
